In [63]:
import os
import sys
path=!pwd
sys.path.append(os.path.join(path[0], '..'))
import numpy as np
import math
import pandas as pd
import re
from datetime import date, datetime, timedelta
import json
from abc import ABCMeta, abstractmethod
import boto3
import snowflake.connector
from io import StringIO
import logging 

logger = logging.getLogger()
logging.basicConfig(level=logging.INFO)
logger.info(f'Starting Notebook')

from utils import *

class Utils():
    @staticmethod
    def to_csv_s3(content, bucket, key_path, filename):
        client = boto3.client('s3')
        key = os.path.join(key_path, filename)
        csv_buffer = StringIO()
        content.to_csv(csv_buffer)
        client.put_object(Bucket=bucket, Key=key, Body=csv_buffer.getvalue())
        logger.info(f'Saved to {bucket}/{key}')
    
    @staticmethod
    def to_pkl_s3(content, bucket, key_path, filename):
        client = boto3.client('s3')
        key = os.path.join(key_path, filename)
        obj = pickle.dumps(content)
        client.put_object(Bucket=bucket, Key=key, Body=obj)
        logger.info(f'Saved model to {os.path.join(bucket, key)}')
        logger.info(f'Saved to {bucket}/{key}')

    @staticmethod
    def read_csv_s3(bucket, key_path,filename):
        client = boto3.client('s3')
        key = os.path.join(key_path, filename)
        obj = client.get_object(Bucket=bucket, Key=key)
        df = pd.read_csv(obj['Body'], na_values="\\N")
        logger.info(f'Read from {bucket}/{key}')
        return df
        
    @staticmethod
    def read_pkl_s3(bucket, key_path,filename):
        client = boto3.client('s3')
        key = os.path.join(key_path, filename)
        obj = client.get_object(Bucket=bucket, Key=key)
        body = obj['Body'].read()
        model = pickle.loads(body)
        logger.info(f'Read from {bucket}/{key}')
        return model    

class Credentials(metaclass=ABCMeta):
    pass
    
    
class SSMPSCredentials(Credentials):
    def __init__(self, secretid: str):
        self._secretid = secretid
        self._secrets = {}
        
    def get_keys(self):
        """
        credential fetching 
        """
        _aws_sm_args = {'service_name': 'secretsmanager', 'region_name': 'us-east-1'}
        secrets_client = boto3.client(**_aws_sm_args)
        get_secret_value_response = secrets_client.get_secret_value(SecretId=self._secretid)
        return get_secret_value_response
    
    
class BaseConnector(metaclass=ABCMeta):
    @abstractmethod
    def connect(self):
        raise NotImplementedError

        
class SnowflakeConnector(BaseConnector):
    def __init__(self, credentials: Credentials):
        keys = credentials.get_keys()
        self._secrets = json.loads(keys.get('SecretString', "{}"))

    def connect(self, dbname: str, schema: str = 'DEFAULT'):
        ctx = snowflake.connector.connect(
            user=self._secrets['login_name'],
            password=self._secrets['login_password'],
            account=self._secrets['account'],
            warehouse=self._secrets['warehouse'],
            database=dbname,
            schema=schema
        )
        return ctx

    
def run_query(querystr, ctx):
    cursor_list = ctx.execute_string(
        querystr
        )
    df = pd.DataFrame.from_records(cursor_list[-1].fetchall(), columns=[x[0] for x in cursor_list[-1].description])
    df.columns= df.columns.str.lower()
    return df

import plotly.express as px
def get_simple_plot(df_plt, x, y, grpby, text, title=''):
    if title=='':
        title = f'{y} vs {x}'
    df_plt[grpby] = df_plt[grpby].astype(str)
    fig = px.line(df_plt,
                  x=x, 
                  y=y, 
                  title=title,
                  color=grpby, 
                  hover_data=[text],
                  width=800, height=400)
    fig.show()
    return 


## Credentials
SF_CREDS = 'datascience-max-dev-sagemaker-notebooks'

## Snowflake connection 
conn=SnowflakeConnector(SSMPSCredentials(SF_CREDS))
ctx=conn.connect("MAX_PROD","DATASCIENCE_STAGE")
cur = ctx.cursor()

input_bucket="hbo-ingest-datascience-content-dev"
output_bucket="hbo-outbound-datascience-content-dev"
key_path = 'cost_allocation/dev'

INFO:root:Starting Notebook
Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
INFO:snowflake.connector.connection:Snowflake Connector for Python Version: 2.7.4, Python Version: 3.6.13, Platform: Linux-4.14.322-170.535.amzn1.x86_64-x86_64-with-glibc2.9
INFO:snowflake.connector.connection:This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.


In [189]:
# df_tif = pd.read_csv('US CROI import_tiffany_mod.csv')
# df_fin = pd.read_csv('df_concat_fin.csv')

# # display(df_tif.head())
# # display(df_fin.head())

# df_tif = df_tif.merge(df_fin[['search_title','legacy_hbomax_series_viewable_id']], on=['search_title'])
# df_tif['TITLE_ID'] = df_tif['legacy_hbomax_series_viewable_id'] 
# df_tif = df_tif[['Cost_ID', 'TITLE_ID', 'TITLE', 'SEASON_LEVEL',
#        'SEASON_NUMBER', 'SEASON_FANOUT', 'MAPPING_TYPE', 'SOURCE_TITLE_NAME',
#        'CONTENT_TYPE', 'STUDIO', 'STATUS', 'AVAIL', 'REMOVAL_DATE',
#        'PROJECT_ID', 'KIDS_FAMILY', 'HUB', 'AIR_YEAR', 'MAX_SERIES_ID',
#        'MAX_MAPPING_TYPE', 'LOAD_TIMESTAMP', 'MAPPING_TYPE_TEMP',
#        'Assigned Person']]

# df_tif.to_csv('df_automated.csv')

In [39]:
df = pd.read_csv('cost_mapping/US CROI import_full.csv')


def extract_first_three_words(text):
    words = text.split()
    if len(words) >= 2 and len(words) >= 4:
        word=' '.join(words[:2])
    elif len(words) >= 4 and len(words) <=6:
        word=' '.join(words[:3])
    elif len(words) >= 7:
        word=' '.join(words[:4])
    else:
        word=text
    return word

def remove_words_starting_with_b(text):
    return ' '.join(word for word in text.split() if not word.startswith('('))

# Apply the function to create the 'search_title' column
df['SOURCE_TITLE_NAME'] = df['SOURCE_TITLE_NAME'].astype(str)
df['search_title'] = df['SOURCE_TITLE_NAME'].str.replace(', The', '', case=False)
df['search_title'] = df['search_title'].str.replace('S1', '', case=False)
df['search_title'] = df['search_title'].str.replace('S2', '', case=False)
df['search_title'] = df['search_title'].str.replace('S3', '', case=False)
df['search_title'] = df['search_title'].str.replace('S4', '', case=False)

df['search_title'] = df['search_title'].apply(extract_first_three_words)
df['search_title'] = df['search_title'].apply(remove_words_starting_with_b)
df.head(20)

df.to_csv('cost_mapping/US CROI import_full_st.csv')

In [ ]:
jupyter-notebooks/cost_mapping/US CROI import_full_ .csv

In [129]:
df = pd.read_csv('cost_mapping/US CROI import_full_.csv')
# df = df[df['Assigned Person'].isin(['Harshitha', 'Mei-Cheng'])]
df = df.drop_duplicates(subset=['SOURCE_TITLE_NAME'])


In [142]:
df_list=[]

for index, row in df[:10].iterrows():
    
    var = row['search_title']
    full_title = row['SOURCE_TITLE_NAME']
    
    try:
        query = f"SET var = '{var}'"
        cur.execute(query)

        title_query=  f'''select 
        distinct series_title_long
        ,rad.legacy_hbomax_series_viewable_id
        , release_year
        , content_type
        from    "INT_DAI_PROD_SHARE"."CONTENT_METADATA_GOLD"."REPORTING_ASSET_DIM_COMBINED" rad
        join    "INT_DAI_PROD_SHARE"."CONTENT_METADATA_GOLD"."REPORTING_ASSET_OFFERING_DIM_COMBINED" raod
        on      rad.legacy_hbomax_viewable_id = raod.legacy_hbomax_viewable_id
        where   rad.asset_type = 'FEATURE'
        and     contains (lower(rad.series_title_long), lower($var)) '''
    
#     cur.execute(title_query)

        df_r =  run_query(title_query, ctx)
    
        if df_r.shape[0]==0:
            columns=['series_title_long', 'legacy_hbomax_series_viewable_id', 'release_year',
            'content_type', 'search_title', 'source_title_name']
            values = [[np.nan], [np.nan], [np.nan], [np.nan], [var], [full_title]]
            df_r = pd.DataFrame(dict(zip(columns, values)))

        else:
            df_r['search_title'] = var
            df_r['source_title_name'] = full_title
            
        print(df_r)

        df_list.append(df_r)

        
    except:
        pass
    
# df_final = pd.concat(df_list)
# df_final

INFO:snowflake.connector.cursor:query: [SET var = 'Slave Play']
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:query: [select distinct series_title_long ,rad.legacy_hbomax_series_viewable_id , releas...]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:query: [SET var = 'Smash!']
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:query: [select distinct series_title_long ,rad.legacy_hbomax_series_viewable_id , releas...]


   series_title_long  legacy_hbomax_series_viewable_id  release_year  \
0                NaN                               NaN           NaN   

   content_type search_title source_title_name  
0           NaN   Slave Play        Slave Play  


INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:query: [SET var = 'Smash!']
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:query: [select distinct series_title_long ,rad.legacy_hbomax_series_viewable_id , releas...]


   series_title_long  legacy_hbomax_series_viewable_id  release_year  \
0                NaN                               NaN           NaN   

   content_type search_title source_title_name  
0           NaN       Smash!         Smash! S1  


INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:query: [SET var = 'Smash!']
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:query: [select distinct series_title_long ,rad.legacy_hbomax_series_viewable_id , releas...]


...

   series_title_long  legacy_hbomax_series_viewable_id  release_year  \
0                NaN                               NaN           NaN   

   content_type  search_title source_title_name  
0           NaN  Sopranos Doc      Sopranos Doc  


INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:query: [SET var = 'Steve Way Special']
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:query: [select distinct series_title_long ,rad.legacy_hbomax_series_viewable_id , releas...]


   series_title_long  legacy_hbomax_series_viewable_id  release_year  \
0                NaN                               NaN           NaN   

   content_type search_title source_title_name  
0           NaN  Soundtracks       Soundtracks  


INFO:snowflake.connector.cursor:query execution done


   series_title_long  legacy_hbomax_series_viewable_id  release_year  \
0                NaN                               NaN           NaN   

   content_type       search_title  source_title_name  
0           NaN  Steve Way Special  Steve Way Special  


In [143]:
df_r

,series_title_long,legacy_hbomax_series_viewable_id,release_year,content_type,search_title,source_title_name
0,NaN,NaN,NaN,NaN,Steve Way Special,Steve Way Special


In [138]:
duplicates = df_final[df_final.duplicated(subset='legacy_hbomax_series_viewable_id', keep=False)]
df_d = pd.DataFrame(duplicates)
df_f = df_final[~df_final.legacy_hbomax_series_viewable_id.isin(list(df_d.legacy_hbomax_series_viewable_id.unique()))]

print(df_d.shape, df_f.shape, df_empty.shape)

(0, 6) (1926, 6) (194, 24)


In [146]:
df_final = df_final.drop_duplicates(subset=['legacy_hbomax_series_viewable_id'])
df_final = df_final.rename(columns={'series_title_long': 'asset_dim_series_title_long', 'search_title':'search_term'})
df_final= df_final[['source_title_name','search_term','asset_dim_series_title_long','legacy_hbomax_series_viewable_id','release_year','content_type']].sort_values(by='source_title_name')

df_final = df_final.sort_values(by=['source_title_name'])
df_final.to_csv('cost_mapping/df_snowflake_search_full.csv')

In [141]:
df_final[df_final.source_title_name.isnull()]

,source_title_name,search_term,asset_dim_series_title_long,legacy_hbomax_series_viewable_id,release_year,content_type


In [105]:
df_final.head(20)

,source_title_name,series_title_long,search_title,legacy_hbomax_series_viewable_id,release_year,content_type
0,101 Places to Get F Up,101 Places to Party Before You Die,101 Places,GYw-6_w6fq0TDwwEAAAhq,2022.0,episode
0,A Dark Room In The Glitterball City,Down a Dark Hall,A Dark,GW_L0QwKaJL3DwgEAAADh,2018.0,None
0,A HAUNTED HOUSE,A Haunted House 2,A HAUNTED HOUSE,GYF5TyQc4RBDDSgEAAACx,2014.0,None
0,A Knight of the Seven Kingdoms S1,A Knight's Tale,A Knight,GYoejpgibYMJUwgEAAAAC,2001.0,None
0,A TINY AUDIENCE: GIANMARCO,A Tiny Audience: Piso 21,A TINY,GXnU7EAmZeoCgwwEAAAXU,2020.0,None
1,A TINY AUDIENCE: GIANMARCO,A Tiny Audience: Alejandra Guzman,A TINY,GXnU6jQIOvJI9wwEAAAU8,2020.0,None
2,A TINY AUDIENCE: GIANMARCO,A Tiny Audience,A TINY,GXcsTHA88FJuAuwEAAASx,2021.0,episode
4,A TINY AUDIENCE: GIANMARCO,A Tiny Audience: Draco Rosa,A TINY,GXnU6jATGyoCgwwEAAAXH,2020.0,None
5,A TINY AUDIENCE: GIANMARCO,A Tiny Audience: Danny Ocean,A TINY,GXnU66ALanqFYqQEAAAVZ,2020.0,None
7,A TINY AUDIENCE: GIANMARCO,A Tiny Audience: Jackie Cruz,A TINY,GXnU7LwbqP5I9wwEAAAVP,2020.0,None


In [22]:


df_empty = df[(~df.search_title.isin(list(df_final.search_title.unique().tolist()))) & (~df.search_title.isin(fin_list))]
df_empty.shape
df_empty[:100]

,Unnamed: 0,Cost_ID,TITLE_ID,TITLE,SEASON_LEVEL,SEASON_NUMBER,SEASON_FANOUT,MAPPING_TYPE,SOURCE_TITLE_NAME,CONTENT_TYPE,STUDIO,STATUS,AVAIL,REMOVAL_DATE,PROJECT_ID,KIDS_FAMILY,HUB,AIR_YEAR,MAX_SERIES_ID,MAX_MAPPING_TYPE,LOAD_TIMESTAMP,MAPPING_TYPE_TEMP,Assigned Person,search_title
1,4949,HBO-OP-000935,NaN,NaN,NaN,NaN,NaN,NaN,Gorilla and the Bird S1,Drama Series Original,NaN,Uncommitted,NaN,NaN,WCF-10019901,No,HBO OP,NaN,NaN,NaN,11/3/23,0,Tiffany,Gorilla and
11,4987,HBO-OP-000849,NaN,NaN,NaN,NaN,NaN,NaN,Great Photo Lovely Life,Documentary Feature Original,NaN,Committed,NaN,NaN,WCF-10023087,No,HBO OP,2023.0,NaN,NaN,11/3/23,0,Tiffany,Great Photo Lovely Life
22,4998,MAX-OP-000987,NaN,NaN,NaN,NaN,NaN,NaN,Greatest Space S1,"Development, Overall Deals, Other",NaN,Committed,NaN,NaN,DTC1034,No,Max OP,2021.0,NaN,NaN,11/3/23,0,Tiffany,Greatest Space
50,5048,MAX-AQ-007444,NaN,NaN,NaN,NaN,NaN,NaN,Greenlight German Genius - HBOMax,International Content Original,HBO INT. - SERIES,Committed,5/23/23,NaN,NaN,No,Max Acquired,2023.0,NaN,NaN,11/3/23,0,Tiffany,Greenlight German
64,5122,MAX-OP-000158,NaN,NaN,NaN,NaN,NaN,NaN,HA Comedy Festival #2,Special Original,NCP,Committed,NaN,NaN,DTC1242,No,Max OP,2021.0,NaN,NaN,11/3/23,0,Tiffany,HA Comedy
66,5129,Do not use 455,NaN,NaN,NaN,NaN,NaN,NaN,HACKERVILLE (EPS 1-6),Movies Acquired,INTERCOMPANY,Committed,7/1/19,NaN,NaN,NaN,HBO Acquired,2019.0,NaN,NaN,11/3/23,0,Tiffany,HACKERVILLE
76,5149,Do not use 3159,NaN,NaN,NaN,NaN,NaN,NaN,HALLOWEEN H2O: 20 YEARS LATER,Movies Acquired,MIRAMAX,Committed,10/1/25,NaN,NaN,NaN,HBO Acquired,2025.0,NaN,NaN,11/3/23,0,Tiffany,HALLOWEEN H2O
78,5154,Do not use 3160,NaN,NaN,NaN,NaN,NaN,NaN,HALLOWEEN: RESURRECTION,Movies Acquired,MIRAMAX,Committed,10/1/25,NaN,NaN,NaN,HBO Acquired,2025.0,NaN,NaN,11/3/23,0,Tiffany,"HALLOWEEN, RESURRECTION"
79,5155,Do not use 3158,NaN,NaN,NaN,NaN,NaN,NaN,HALLOWEEN: THE CURSE OF MICHAEL MYERS,Movies Acquired,MIRAMAX,Committed,10/1/25,NaN,NaN,NaN,HBO Acquired,2025.0,NaN,NaN,11/3/23,0,Tiffany,CURSE OF MICHAEL MYERS
81,5157,Do not use 2898,NaN,NaN,NaN,NaN,NaN,NaN,HANG EM HIGH,Movies Acquired,MGM,Committed,7/1/23,NaN,NaN,NaN,HBO Acquired,2023.0,NaN,NaN,11/3/23,0,Tiffany,HANG EM HIGH


In [5]:
df_concat= pd.read_csv('df_concat.csv')
df_concat = df_concat.sort_values(by=['series_title_long'])

df_concat.head(100)

,Unnamed: 0,series_title_long,legacy_hbomax_series_viewable_id,release_year,content_type,search_title
195,56,99 Homes,GYfu6xAj5b3IkwgEAAAAD,2014,NaN,Home
84,0,A Guy Named Joe,GXt509QQlasPCwgEAAAma,1943,NaN,Guy Named Joe
160,5,A Prairie Home Companion,GXfuVlQz1mJWdkQEAAAfJ,2006,NaN,Home
217,95,"Acasa, My Home",GYWfIagAFZbORYAEAAAHT,2020,NaN,Home
43,65,Acoustic Home,GYZaDUwRqMDB0bAEAAABP,2022,NaN,Home
391,152,Adventures Italian Style,GYfpzqAK18sMRRQEAAACj,2020,NaN,It
302,39,Agent Hamilton: But Not If It Concerns Your Da...,GYcIF4wjDLXconAEAAAAJ,2012,NaN,It
159,4,Alive Day Memories: Home From Iraq,GVU2z_wAlGo7DwvwIAVsE,2007,movie,Home
30,1,All American: Homecoming,GYgR3zAxugZJAsQEAAAA6,2022,NaN,Home
138,4,An American Haunting,GV7tzEQnWKYORngEAAACL,2006,NaN,Haunting


(460, 5)

In [109]:
# df_d = df_d.drop_duplicates(subset=['legacy_hbomax_series_viewable_id'])
# df_d = df_d.drop(df_d[df_d['legacy_hbomax_series_viewable_id'] == 'EdhwKKy5rDwwEAAACK'].index)
# df_d = df_d.drop(df_d[df_d['legacy_hbomax_series_viewable_id'] == 'GYgYXoQjs2FpXwgEAAAEl'].index)
# df_d = df_d.drop(df_d[df_d['legacy_hbomax_series_viewable_id'] == 'GYU5yzQjrEMLDwwEAAAAx'].index)

# df_f = df_f.drop(df_f[df_f['legacy_hbomax_series_viewable_id'] == 'GXXAljgW4WcPCwgEAAAHP'].index)
# df_f = df_f.drop(df_f[df_f['legacy_hbomax_series_viewable_id'] == 'GYKa2lwaD8ZzDwgEAAABY'].index)
# df_f = df_f.drop(df_f[df_f['legacy_hbomax_series_viewable_id'] == 'GWGwmGgmm95yXCQEAAAB2'].index)
# df_f = df_f.drop(df_f[df_f['legacy_hbomax_series_viewable_id'] == 'GXYJVxQDWUKnCVgEAAAdS'].index)


,series_title_long,legacy_hbomax_series_viewable_id,release_year,content_type,search_title
0,Gordita Chronicles,GYnl_JQsLosPCwgEAAACs,2022,None,Gordita Chronicles
0,Gotham Knights,GY_98Sg0MQbiWtQEAAAL3,2023,episode,Gotham Knights
0,Green Lantern,GXXAljgW4WcPCwgEAAAHP,2011,None,Green Lantern
1,Green Lantern,GX07YeQv6q7CttQEAAAF_,2011,movie,Green Lantern
0,The Legend of Tarzan,GYKa2lwaD8ZzDwgEAAABY,2016,None,The Legend of Tarzan
1,The Legend of Tarzan,GWGwmGgmm95yXCQEAAAB2,2016,None,The Legend of Tarzan
0,Guys and Dolls,GX5LyvQrA8YuLwgEAAAAg,1955,None,Guys And Dolls
0,Habla Loud,GYwcGdQcWFV6lLQEAAAAC,2022,movie,HABLA LOUD
0,Hackerville,GXO6ZlQ4ph7ZzlAEAAAAa,2019,None,HACKERVILLE
0,Hail Satan?,GY4d1MwS_i6-1mgEAAALQ,2019,None,HAIL SATAN?


,Unnamed: 0,Cost_ID,TITLE_ID,TITLE,SEASON_LEVEL,SEASON_NUMBER,SEASON_FANOUT,MAPPING_TYPE,SOURCE_TITLE_NAME,CONTENT_TYPE,STUDIO,STATUS,AVAIL,REMOVAL_DATE,PROJECT_ID,KIDS_FAMILY,HUB,AIR_YEAR,MAX_SERIES_ID,MAX_MAPPING_TYPE,LOAD_TIMESTAMP,MAPPING_TYPE_TEMP,Assigned Person,search_title
189,5472,MAX-AQ-005507,NaN,NaN,NaN,NaN,NaN,NaN,Hank Zipzer - S1-S3 - HBOMax - K&F,Kids Content Acquired,DHX WORLDWIDE (U) LIMITED (INT'L - 0%) - #2,Committed,5/13/22,NaN,NaN,Yes,Max Acquired,2022.0,NaN,NaN,11/3/23,0,Tiffany,Hank Zipzer


In [25]:
title_query=  f'''select  distinct rad.legacy_hbomax_series_viewable_id
from    "INT_DAI_PROD_SHARE"."CONTENT_METADATA_GOLD"."REPORTING_ASSET_DIM_COMBINED" rad
join    "INT_DAI_PROD_SHARE"."CONTENT_METADATA_GOLD"."REPORTING_ASSET_OFFERING_DIM_COMBINED" raod
on      rad.legacy_hbomax_viewable_id = raod.legacy_hbomax_viewable_id
where   rad.asset_type = 'FEATURE'
and     rad.series_title_long = 'Gossip' '''

df =  run_query(title_query, ctx)
display(df.head())

INFO:snowflake.connector.cursor:query: [select  distinct rad.legacy_hbomax_series_viewable_id from    "INT_DAI_PROD_SHAR...]
INFO:snowflake.connector.cursor:query execution done


,legacy_hbomax_series_viewable_id
0,GYl8R2AfZjqmbnwEAAACW


In [9]:
session = ctx.session()


title_query=  '''select  rad.legacy_hbomax_series_viewable_id
from    "INT_DAI_PROD_SHARE"."CONTENT_METADATA_GOLD"."REPORTING_ASSET_DIM_COMBINED" rad
where   rad.asset_type = :variable
limit 1'''



INFO:snowflake.connector.cursor:query: [select  rad.legacy_hbomax_series_viewable_id from    "INT_DAI_PROD_SHARE"."CONTE...]
INFO:snowflake.connector.cursor:query execution done


ProgrammingError: 002049 (42601): 01b01622-0408-8af8-00f8-4106d73c42ef: SQL compilation error: error line 3 at position 25
Bind variable :variable not set.